In [6]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import os

connect_str = input("Azure Storage Connection String: ")

In [7]:
blob_service_client = BlobServiceClient.from_connection_string(connect_str)
container_client = blob_service_client.get_container_client('data')

In [11]:
file_name = 'household_power_consumption.txt'
data_path = '.data'
if not os.path.exists(data_path):
    os.mkdir(data_path)
download_file_path = os.path.join(data_path, file_name)

In [12]:
azure_path = 'power-consumption/data'
blob_client = container_client.get_blob_client(os.path.join(azure_path, file_name))
with open(download_file_path, "wb") as download_file:
    download_file.write(blob_client.download_blob().readall())

In [ ]:
df = pd.read_csv(download_file_path, sep=';', low_memory=False, 
    na_values=['nan','?'], parse_dates={'Date_time':['Date','Time']}, 
    index_col='Date_time', infer_datetime_format=True, header=0)

In [4]:
df['Sub_metering_4'] = (df.iloc[:,0] * 1000 / 60) - (df.iloc[:,4] + df.iloc[:,5] + df.iloc[:,6])

In [5]:
processed_path = '.processed'
if not os.path.exists(processed_path):
    os.mkdir(processed_path)
df.to_csv(os.path.join(processed_path, 'household-pow.csv'))